### Getting started with PyOphidia

This notebook provides some basic examples of how to use the ophidia framework features for climate data analysis and, in particular, it shows some of the main commands from the PyOphidia module. 

**PyOphidia** is a GPLv3-licensed Python module to interact with the Ophidia framework. It implements two main classes:
   
- **Client** class: it supports the submissions of Ophidia commands and workflows as well as the management of sessions from Python code (similar to the Ophidia Terminal)
   - It allows running all the Ophidia operators, including massive tasks and workflows
   
- **Cube** class: it builds on the client class and provides the datacube type abstraction and the methods to manipulate, process and get information on cubes objects and 
   - It defines a object-oriented approach to handle the datacubes more naturally
   
While the *cube* module provides a user-friendly interface, the *client* module allows a finer specification of the operators.

First of all import PyOphidia modules

In [ ]:
from PyOphidia import cube, client

Then, setup a connection to the server instance. Connection details can be automatically inferred from the environment.

In [ ]:
cube.Cube.setclient(read_env=True)

Let's now load a NetCDF file. We can inspect the file with the *explorenc* Ophidia operator

In [ ]:
from os.path import expanduser
home = expanduser("~")

input_file=home+"/data/CMIP6/ScenarioMIP/CMCC/CMCC-CM2-SR5/ssp585/r1i1p1f1/Amon/tas/gn/v20200622/tas_Amon_CMCC-CM2-SR5_ssp585_r1i1p1f1_gn_201501-210012.nc"

cube.Cube.explorenc(src_path=input_file)

We can now create a datacube from the NetCDF file:
- The file is located in the home folder under **data/CMIP6/ScenarioMIP/CMCC/CMCC-CM2-SR5/ssp585/r1i1p1f1/Amon/tas/gn/v20200622/tas_Amon_CMCC-CM2-SR5_ssp585_r1i1p1f1_gn_201501-210012.nc**
- The variable to be imported is **tas**
- Data should be arranged in order to operate on time series (**time** dimension) 

In [ ]:
%%time
mycube = cube.Cube.importnc2(
                src_path=input_file,
                measure='tas',
                imp_dim='time',
                ncores=1,
                nfrag=1,
                description="Imported cube"
        )

Inspect the cube and its dimensions structure

In [ ]:
mycube.info()

Check the datacubes available in the virtual file system.

In [ ]:
cube.Cube.list(level=2)

Subset the datacube over time

**Note: each instance method produces a new datacube object**

In [ ]:
mycube2 = mycube.subset(
                subset_dims="time",
                subset_filter="1:60",
                subset_type="index",
                time_filter="yes",
                ncores=1,
                description="Subsetted cube"
        )

We can use the explore method to check the content of the datacube. 

In [ ]:
mycube2.explore(limit_filter=1)

What if we want to explore just a specific portion of the datacube?

In [ ]:
mycube2.explore(subset_dims="lat|lon|time",subset_type="index",subset_filter="1:2|1:4|1:4")

Compute the **maximum** value over the time series for each point in the spatial domain. We can also compute other metrics (see http://ophidia.cmcc.it/documentation/users/operators/OPH_REDUCE.html)

In [ ]:
mycube3 = mycube2.reduce(
    operation='max',
    ncores=1,
    description="Reduced cube"
)

Let's export the data into a Python-friendly structure. 

**Note: this is the first time we move data from the server-side to the Notebook**

The structure looks something like this

<img src="imgs/export_array.png" alt="Export Array" width="600">


In [ ]:
data = mycube3.export_array()

from IPython.lib.pretty import pprint
pprint(data['dimension'])

The data exported in the Python structure can be used to create a map (note the definition of a Python function)

In [ ]:
%matplotlib inline

def plotData(data):
    
    import cartopy.crs as ccrs
    import matplotlib.pyplot as plt
    from cartopy.mpl.geoaxes import GeoAxes
    from cartopy.util import add_cyclic_point
    import numpy as np
    import warnings
    warnings.filterwarnings("ignore")

    fig = plt.figure(figsize=(12, 6), dpi=100)

    #Add Geo axes to the figure with the specified projection (PlateCarree)
    projection = ccrs.PlateCarree()
    ax = plt.axes(projection=projection)

    #Draw coastline and gridlines
    ax.coastlines()

    gl = ax.gridlines(crs=projection, draw_labels=True, linewidth=1, color='black', alpha=0.9, linestyle=':')
    gl.xlabels_top = False
    gl.ylabels_right = False

    lat = data['dimension'][0]['values'][ : ]
    lon = data['dimension'][1]['values'][ : ]
    var = data['measure'][0]['values'][ : ]
    var = np.reshape(var, (len(lat), len(lon)))

    #Wraparound points in longitude
    var_cyclic, lon_cyclic = add_cyclic_point(var, coord=np.asarray(lon))
    x, y = np.meshgrid(lon_cyclic,lat)

    #Define color levels for color bar
    clevs = np.arange(200,340,2)

    #Set filled contour plot
    cnplot = ax.contourf(x, y, var_cyclic, clevs, transform=projection,cmap=plt.cm.jet)
    plt.colorbar(cnplot,ax=ax)

    ax.set_aspect('auto', adjustable=None)

    plt.title('Maximum Near-Surface Air Temperature (deg K)')
    plt.show()
    
plotData(data)

Our workspace now contains several datacubes from the experiments just run.

In [ ]:
cube.Cube.list(level=2)

Once done, we can clear the space before moving to other notebooks.

In [ ]:
cube.Cube.deletecontainer(container='tas_Amon_CMCC-CM2-SR5_ssp585_r1i1p1f1_gn_201501-210012.nc',force='yes')

The virtual file system should now be "clean"

In [ ]:
cube.Cube.list(level=2)